TRY
top_k_categorical_accuracy
https://stackoverflow.com/questions/47887533/keras-convolution-along-samples
https://keras.io/layers/wrappers/#timedistributed
### !Try target position time (current or next)

In [1]:
input_width = 160
input_height = 100
sequence_size = 15
class_number = 12
channels = 3
data_path = "D:\\Python\\Keras\\Wormax\\data_prepared\\"
model_name = 'models//worm_sequence15_4.h5'
import keras
import numpy as np

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import models, optimizers, layers
import keras.backend as K
from keras.applications import Xception
from keras.layers import TimeDistributed, Conv2D, Dropout, LSTM, Dense, MaxPooling2D, Flatten, GRU

def actual_acc(y_true, y_pred):
    return K.equal(K.argmax(y_pred), K.argmax(y_true))

def convolution_feature_extractor(input_height, input_width):
    model = models.Sequential()        
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(input_height, input_width, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    
    model.summary()    
    return model

def define_model():
    
    model = models.Sequential()
    
    model.add(TimeDistributed(
            convolution_feature_extractor(input_height, input_width),
            input_shape=(None, input_height, input_width, 3)
            ))

    model.add(GRU(128, return_sequences=True))#, dropout=0.5))
    model.add(GRU(128))
    model.add(Dense(class_number, activation='softmax'))    
    
    model.compile(optimizer=optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=[actual_acc])

    model.summary()
    return model

In [3]:
model = define_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 158, 32)       896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 38, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 36, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 18, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 16, 128)        147584    
__________

In [4]:
# little prepocessing
from math import atan2, pi

def get_angle(x, y):
    return atan2(y, x)

def get_direction(x, y, n_classes = 12):
    return round(get_angle(x, y)/2/pi*n_classes)%n_classes

In [5]:
import os
import random
from functools import reduce
from keras.utils import to_categorical

# Training and validation
data_ratio = 0.7
            
# Generator done for not to overflow MEM
# holds one data file for every instance(train and validation)
def generator(data_dir, sequence_size, num_classes, role, batch_size=128, shuffle=True):
    
    listdir = []
    listdir = filter(lambda x: os.path.isfile, os.listdir(data_dir))
    listdir = np.array(list(listdir))
    if shuffle:
        random.shuffle(listdir)
    
    #print('Found {} files for {}'.format(len(listdir), role))
    
    file_i = 0
    while 1:
        arr = np.load(data_dir + listdir[file_i])
        file_i = (file_i+1) if file_i+1<len(listdir) else 0
        
        # Expanding blocks and banning inappropriate
        data = []
        banned_indexes = np.array([])
        for i in arr:
            ban = np.arange(len(data)+len(i)-sequence_size, len(data)+len(i)+1)
            banned_indexes = np.concatenate((banned_indexes, ban), axis=0)
            for j in i:
                data.append(j)
        data = np.array(data)        
        
        if role == 'train':
            data = data[:int(round(len(data)*data_ratio))]
        elif role == 'validation':
            data = data[int(round(len(data)*data_ratio)):]
        else:
            raise TypeError('bad role parameter')
        
        indexes = np.arange(len(data)-sequence_size-1)
        indexes = np.delete(indexes, banned_indexes)
        if shuffle:
            np.random.shuffle(indexes)
        
        for i in range(0, len(indexes), batch_size):
            samples = np.zeros((batch_size, sequence_size, input_height, input_width, channels))
            targets = np.zeros((batch_size, num_classes))
            for j, index in enumerate(indexes[i:i + batch_size]):
                # some issue with shapes(dummy reshape)
                sample = np.zeros((sequence_size, input_height, input_width, channels))
                for k, dt in enumerate(data[index:index + sequence_size, 0]):
                    sample[k] = dt[0]
                samples[j] = sample
                targets[j] = to_categorical(get_direction(*data[index + sequence_size - 1][1][:2]), num_classes=num_classes)
            
            # will not work without this
            samples = samples / 255
            
            yield samples, targets

In [6]:
train_generator = generator(data_path, sequence_size, class_number, 'train', batch_size=10)
validation_generator = generator(data_path, sequence_size, class_number, 'validation', batch_size=10)

print(next(train_generator)[0].shape)
print(next(train_generator)[1].shape)

(10, 15, 100, 160, 3)
(10, 12)


Q:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
Q:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


In [7]:
if False:
    %matplotlib notebook
    import cv2
    import matplotlib.pyplot as plt
    from matplotlib.animation import FuncAnimation
    from grabscreen import grab_screen
    from image_preproc import preproc_img, prepare_image

    gridsize = (1, 1)
    fig = plt.figure(figsize=(6, 4))
    ax1 = plt.subplot2grid(gridsize, (0, 0))

    samples, targets = next(train_generator)
    im1 = ax1.imshow(samples[0][0])

    n = 8
    fig.suptitle(str(np.argmax(targets[n])), fontsize=20)
    def update(i):
        i %= sequence_size
        im1.set_data(samples[n][i])

    ani = FuncAnimation(plt.gcf(), update, interval=50)
    plt.show()

In [8]:
#####9####
###8###10##
##7#####11#
#6#######0#
##5#####1##
###4###2###
#####3####

In [ ]:
# count class instances count for balancing
if False:
    i = 0
    classes = np.zeros((class_number))
    for samples, targets in generator(data_path, sequence_size, class_number, 'train', batch_size=128):
        for j in targets:
            classes += j
        i += 1
        if i == 1500:
            break
    print(classes)

### tensorboard --logdir=D:\Python\Keras\Wormax\log_dir

In [ ]:
class_weight = {0: 1.14,
                 1: 1.12,
                 2: 1.21,
                 3: 1.36,
                 4: 1.35,
                 5: 1.16,
                 6: 1.0,
                 7: 1.02,
                 8: 1.04,
                 9: 1.1,
                 10: 1.15,
                 11: 1.24}

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='log_dir\\' + model_name
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_name + '.h5',
        monitor='val_loss',
        save_best_only=True,
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='loss', 
        factor=0.5,                              
        patience=50, 
        min_lr=0.00001
    )
]

steps_per_epoch = 100
history = model.fit_generator(train_generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=500,
                            validation_data=validation_generator,
                            validation_steps=int(round(steps_per_epoch/data_ratio*(1-data_ratio))),
                            shuffle=True,
                            class_weight=class_weight,
                            callbacks=callbacks
                        )

Epoch 1/500


Q:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: using a non-integer array as obj in delete will result in an error in the future
Q:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:44: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


100/100 [==============================] - 32s 316ms/step - loss: 2.7362 - actual_acc: 0.1460 - val_loss: 2.4758 - val_actual_acc: 0.1512
Epoch 2/500
100/100 [==============================] - 26s 260ms/step - loss: 2.3958 - actual_acc: 0.2710 - val_loss: 2.5087 - val_actual_acc: 0.1326
Epoch 3/500
100/100 [==============================] - 26s 259ms/step - loss: 1.9142 - actual_acc: 0.4010 - val_loss: 2.5215 - val_actual_acc: 0.1605
Epoch 4/500
100/100 [==============================] - 26s 261ms/step - loss: 1.6687 - actual_acc: 0.4960 - val_loss: 2.5620 - val_actual_acc: 0.1651
Epoch 5/500
100/100 [==============================] - 28s 280ms/step - loss: 1.4672 - actual_acc: 0.5320 - val_loss: 2.8667 - val_actual_acc: 0.2047
Epoch 6/500
100/100 [==============================] - 26s 263ms/step - loss: 1.1580 - actual_acc: 0.6270 - val_loss: 3.2837 - val_actual_acc: 0.1581
Epoch 7/500
100/100 [==============================] - 30s 302ms/step - loss: 1.9433 - actual_acc: 0.4210 - val_

100/100 [==============================] - 29s 293ms/step - loss: 0.9325 - actual_acc: 0.7050 - val_loss: 2.8876 - val_actual_acc: 0.2116
Epoch 56/500
100/100 [==============================] - 26s 265ms/step - loss: 2.2130 - actual_acc: 0.3870 - val_loss: 2.1767 - val_actual_acc: 0.2721
Epoch 57/500
100/100 [==============================] - 26s 264ms/step - loss: 1.6969 - actual_acc: 0.4820 - val_loss: 2.2399 - val_actual_acc: 0.2419
Epoch 58/500
100/100 [==============================] - 26s 264ms/step - loss: 1.3526 - actual_acc: 0.5810 - val_loss: 2.3685 - val_actual_acc: 0.2465
Epoch 59/500
100/100 [==============================] - 27s 265ms/step - loss: 1.2281 - actual_acc: 0.6070 - val_loss: 2.3209 - val_actual_acc: 0.2512
Epoch 60/500
100/100 [==============================] - 32s 317ms/step - loss: 1.1069 - actual_acc: 0.6470 - val_loss: 2.5202 - val_actual_acc: 0.2488
Epoch 61/500
100/100 [==============================] - 27s 265ms/step - loss: 1.0661 - actual_acc: 0.6540 

In [ ]:
from keras import backend as K
K.clear_session()